In [1]:
import tkinter as tk
from tkinter import messagebox
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

def connect_db():
    try:
        conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(
            conexao_bd,
            connect_args={'options': '-csearch_path=sistema,public'}
        )
        conn = engine.connect()
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None
    

    
def buscar_por_cpf(cpf):
    conn = connect_db()  # Conecta ao banco de dados
    if conn:
        try:
            # Busca informações da pessoa pelo CPF
            pessoa_query = text("""
                SELECT p.Nome, p.CPF, e.Rua, e.Numero, e.Bairro, e.Cidade, e.Estado 
                FROM Pessoa p 
                LEFT JOIN Endereco e ON p.ID_Pessoa = e.ID_Pessoa 
                WHERE p.CPF = :cpf
            """)
            pessoa_result = conn.execute(pessoa_query, {"cpf": cpf}).fetchone()

            if pessoa_result:
                # Exibe as informações básicas da pessoa
                nome, cpf, rua, numero, bairro, cidade, estado = pessoa_result
                resultado = f"Nome: {nome}\nCPF: {cpf}\nEndereço: {rua}, {numero}, {bairro}, {cidade}, {estado}\n"

                # Busca emails
                email_query = text("SELECT Email FROM Email WHERE ID_Pessoa = (SELECT ID_Pessoa FROM Pessoa WHERE CPF = :cpf)")
                emails = conn.execute(email_query, {"cpf": cpf}).fetchall()
                resultado += "Emails: " + ", ".join([email[0] for email in emails]) + "\n"

                # Busca telefones
                telefone_query = text("SELECT Telefone FROM Telefone WHERE ID_Pessoa = (SELECT ID_Pessoa FROM Pessoa WHERE CPF = :cpf)")
                telefones = conn.execute(telefone_query, {"cpf": cpf}).fetchall()
                resultado += "Telefones: " + ", ".join([telefone[0] for telefone in telefones]) + "\n"

                # Verifica se a pessoa é médico, cliente ou secretário
                medico_query = text("SELECT CRM, Especializacao FROM Medico WHERE ID_Pessoa = (SELECT ID_Pessoa FROM Pessoa WHERE CPF = :cpf)")
                medico_result = conn.execute(medico_query, {"cpf": cpf}).fetchone()
                if medico_result:
                    crm, especializacao = medico_result
                    resultado += f"Médico\nCRM: {crm}\nEspecialização: {especializacao}\n"

                cliente_query = text("SELECT Convenio FROM Cliente WHERE ID_Pessoa = (SELECT ID_Pessoa FROM Pessoa WHERE CPF = :cpf)")
                cliente_result = conn.execute(cliente_query, {"cpf": cpf}).fetchone()
                if cliente_result:
                    convenio = cliente_result[0]
                    resultado += f"Cliente\nConvênio: {convenio}\n"

                secretario_query = text("SELECT ID_Secretario FROM Secretario WHERE ID_Pessoa = (SELECT ID_Pessoa FROM Pessoa WHERE CPF = :cpf)")
                secretario_result = conn.execute(secretario_query, {"cpf": cpf}).fetchone()
                if secretario_result:
                    # Busca responsabilidades do secretário
                    responsabilidade_query = text("SELECT Descricao FROM Responsabilidade WHERE ID_Secretario = :id_secretario")
                    responsabilidades = conn.execute(responsabilidade_query, {"id_secretario": secretario_result[0]}).fetchall()
                    resultado += "Secretário\nResponsabilidades: " + ", ".join([resp[0] for resp in responsabilidades]) + "\n"
                
                # Exibe o resultado
                text_resultado.config(state='normal')
                text_resultado.delete(1.0, tk.END)  # Limpa o campo de texto
                text_resultado.insert(tk.END, resultado)
                text_resultado.config(state='disabled')
            else:
                messagebox.showerror("Erro", "CPF não encontrado.")
        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro ao buscar o CPF: {e}")
            print(f"Erro ao buscar por CPF: {str(e)}")
        finally:
            conn.close()

# Função para capturar o CPF e realizar a busca
def buscar():
    cpf = entry_cpf.get()
    if cpf:
        buscar_por_cpf(cpf)
    else:
        messagebox.showerror("Erro", "Por favor, insira um CPF válido.")

# Interface gráfica usando Tkinter
root = tk.Tk()
root.title("Consulta por CPF")

# Campo de texto para o CPF
tk.Label(root, text="CPF").grid(row=0, column=0)
entry_cpf = tk.Entry(root)
entry_cpf.grid(row=0, column=1)

# Botão de busca
btn_buscar = tk.Button(root, text="Buscar", command=buscar)
btn_buscar.grid(row=0, column=2)

# Campo de texto para exibir os resultados
text_resultado = tk.Text(root, height=15, width=80, state='disabled')
text_resultado.grid(row=1, column=0, columnspan=3)

# Loop da interface gráfica
root.mainloop()
